# Treinamento de Modelos - Risco de Crédito

Este notebook realiza o treinamento de modelos de classificação e avalia a performance com métricas padrão.

In [9]:
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Carregar dados
dados = pd.read_csv('../dados/dados_credito_tratados.csv')

# Remover coluna ID
dados = dados.drop(columns=['ID'])

# Separar variáveis
X = dados.drop(columns=['inadimplente'])
y = dados['inadimplente']

# Dividir em treino/teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_treino = scaler.fit_transform(X_treino)
X_teste = scaler.transform(X_teste)

# Treinar modelos
modelo_lr = LogisticRegression(solver='saga', penalty='l2', max_iter=10000)
modelo_lr.fit(X_treino, y_treino)

modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_treino, y_treino)

# Avaliar modelos
for nome, modelo in zip(['Random Forest', 'Regressão Logística'], [modelo_rf, modelo_lr]):
    y_pred = modelo.predict(X_teste)
    y_prob = modelo.predict_proba(X_teste)[:, 1]
    print(f'\nModelo: {nome}')
    print(classification_report(y_teste, y_pred))
    print(f'AUC-ROC: {roc_auc_score(y_teste, y_prob):.4f}')

# Criar diretório se não existir
os.makedirs('../resultados/modelos', exist_ok=True)

# Salvar modelo Random Forest
joblib.dump(modelo_rf, '../resultados/modelos/modelo_rf.pkl')

# Salvar scaler (usado para regressão logística, e pode ser útil no app)
joblib.dump(scaler, '../resultados/modelos/escalonador.pkl')
joblib.dump(X.columns, '../resultados/modelos/colunas_treinamento.pkl')


Modelo: Random Forest
              precision    recall  f1-score   support

           0       0.84      0.94      0.89      7040
           1       0.62      0.36      0.46      1960

    accuracy                           0.81      9000
   macro avg       0.73      0.65      0.67      9000
weighted avg       0.79      0.81      0.79      9000

AUC-ROC: 0.7547

Modelo: Regressão Logística
              precision    recall  f1-score   support

           0       0.82      0.97      0.89      7040
           1       0.68      0.23      0.35      1960

    accuracy                           0.81      9000
   macro avg       0.75      0.60      0.62      9000
weighted avg       0.79      0.81      0.77      9000

AUC-ROC: 0.7230


['../resultados/modelos/colunas_treinamento.pkl']